#### 1. Setup & Import Libraries

In [4]:
import pandas as pd
import numpy as np
import re

ACADEMIC_PATH = r'../../data/raw/academic_records.csv'
ADMISSION_PATH = r'../../data/raw/admission.csv'
TEST_PATH = r'../../data/raw/test.csv'
academic_records = pd.read_csv(ACADEMIC_PATH)
admission = pd.read_csv(ADMISSION_PATH)

#### 2. Cleaninng pipeline

##### 2.1 Merge table and process columns

In [2]:
academic_records.head()

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH
0,f022ed8d1ac1,HK2 2020-2021,2.19,2.02,18,18
1,f022ed8d1ac1,HK1 2022-2023,0.95,2.12,14,7
2,f022ed8d1ac1,HK1 2023-2024,0.81,1.89,29,16
3,f022ed8d1ac1,HK2 2022-2023,1.37,1.93,26,23
4,f022ed8d1ac1,HK2 2023-2024,1.71,1.91,16,13


In [17]:
adm = admission.copy()
acad = academic_records.copy()

In [18]:
def parse_semester_string(sem_str):
    """
    Chuyển đổi chuỗi như 'HK1 2023-2024' thành mã số 20231 để sort được.
    Logic: Năm * 10 + Kỳ
    """
    s = str(sem_str).strip()
    # Tìm tất cả các con số trong chuỗi
    digits = re.findall(r'\d+', s)
    
    if len(digits) >= 2:
        # Giả sử số nhỏ là kỳ, số lớn (4 chữ số) là năm
        # Tìm năm (thường là số có 4 chữ số đầu tiên tìm thấy)
        years = [int(d) for d in digits if len(d) == 4]
        sems = [int(d) for d in digits if len(d) == 1]
        
        if years and sems:
            year = years[0]
            sem = sems[0]
            return year * 10 + sem
            
    return 0 

parse_semester_string(academic_records['HOC_KY'][0])

20202

In [19]:
# Tạo cột HOC_KY_INT dùng để sort
acad['HOC_KY_INT'] = acad['HOC_KY'].apply(parse_semester_string)

In [22]:
acad.head()

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,HOC_KY_INT
0,f022ed8d1ac1,HK2 2020-2021,2.19,2.02,18,18,20202
1,f022ed8d1ac1,HK1 2022-2023,0.95,2.12,14,7,20221
2,f022ed8d1ac1,HK1 2023-2024,0.81,1.89,29,16,20231
3,f022ed8d1ac1,HK2 2022-2023,1.37,1.93,26,23,20222
4,f022ed8d1ac1,HK2 2023-2024,1.71,1.91,16,13,20232


In [20]:
# 1. Chuẩn hóa ID
adm['MA_SO_SV'] = adm['MA_SO_SV'].astype(str)
acad['MA_SO_SV'] = acad['MA_SO_SV'].astype(str)

In [21]:
adm['MA_SO_SV'][0]

'0570116c3448'

In [27]:
error_count = (acad['HOC_KY_INT'] == 0).sum()
print(f"Có {error_count} dòng không đọc được HOC_KY.")

Có 0 dòng không đọc được HOC_KY.


In [28]:
 # Merge dữ liệu
df = pd.merge(acad, adm, on='MA_SO_SV', how='left')
# Sắp xếp theo Time-series chuẩn xác dựa trên cột vừa tạo
df = df.sort_values(by=['MA_SO_SV', 'HOC_KY_INT']).reset_index(drop=True)

In [29]:
df

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,HOC_KY_INT,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN
0,00003e092652,HK1 2023-2024,1.64,1.97,18,15,20231,2023,100,A00,21.32,20.25
1,00003e092652,HK2 2023-2024,1.53,2.05,18,13,20232,2023,100,A00,21.32,20.25
2,000e15519006,HK1 2021-2022,3.85,3.85,9,9,20211,2021,1,D07,23.84,22.43
3,000e15519006,HK2 2021-2022,2.77,3.12,19,19,20212,2021,1,D07,23.84,22.43
4,000e15519006,HK1 2022-2023,2.83,2.98,21,21,20221,2021,1,D07,23.84,22.43
...,...,...,...,...,...,...,...,...,...,...,...,...
105721,fffd51317dd2,HK2 2022-2023,0.61,1.78,15,5,20222,2020,1,A00,17.61,16.10
105722,ffff4d891f10,HK1 2022-2023,3.04,3.04,18,18,20221,2022,100,A00,25.98,19.91
105723,ffff4d891f10,HK2 2022-2023,3.16,3.12,18,18,20222,2022,100,A00,25.98,19.91
105724,ffff4d891f10,HK1 2023-2024,2.88,3.00,21,21,20231,2022,100,A00,25.98,19.91


##### 2.2 Data and Logic Process

In [35]:
cols_float = ['GPA', 'CPA', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
cols_int = ['TC_DANGKY', 'TC_HOANTHANH']
    
for col in cols_float:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
for col in cols_int:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Logic: Hoàn thành <= Đăng ký
df['TC_HOANTHANH'] = np.minimum(df['TC_HOANTHANH'], df['TC_DANGKY'])
    
# Chuẩn hoá tỉ lệ hoàn thành tín chỉ 
df['COMPLETION_RATE'] = df['TC_HOANTHANH'] / (df['TC_DANGKY'] + 1e-9)
df['COMPLETION_RATE'] = df['COMPLETION_RATE'].clip(0, 1)

# Clip điểm số
df['GPA'] = df['GPA'].clip(0, 4.0)
df['CPA'] = df['CPA'].clip(0, 4.0)

# thêm Admission Gap Feature
if 'DIEM_TRUNGTUYEN' in df.columns and 'DIEM_CHUAN' in df.columns:
    df['ADMISSION_GAP'] = df['DIEM_TRUNGTUYEN'] - df['DIEM_CHUAN']
    
# Lọc rác
initial_len = len(df)
df = df[df['TC_DANGKY'] > 0]
    

In [33]:
df.head()

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,HOC_KY_INT,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN,COMPLETION_RATE,ADMISSION_GAP
0,00003e092652,HK1 2023-2024,1.64,1.97,18,15,20231,2023,100,A00,21.32,20.25,0.833333,1.07
1,00003e092652,HK2 2023-2024,1.53,2.05,18,13,20232,2023,100,A00,21.32,20.25,0.722222,1.07
2,000e15519006,HK1 2021-2022,3.85,3.85,9,9,20211,2021,1,D07,23.84,22.43,1.000000,1.41
3,000e15519006,HK2 2021-2022,2.77,3.12,19,19,20212,2021,1,D07,23.84,22.43,1.000000,1.41
4,000e15519006,HK1 2022-2023,2.83,2.98,21,21,20221,2021,1,D07,23.84,22.43,1.000000,1.41


In [32]:
df.shape

(105726, 14)

In [ ]:
df.to_csv('processed_data.csv', index=False)